<a href="https://colab.research.google.com/github/phamduyaaaa/Robot-Kinematic/blob/main/DiffRobot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# === LỚP ROBOT VISAI ===
class DifferentialDriveRobot:
    def __init__(self, x=0.0, y=0.0, theta=0.0):
        self.x = x
        self.y = y
        self.theta = theta

    def update(self, v, omega, dt):
        self.x += v * np.cos(self.theta) * dt
        self.y += v * np.sin(self.theta) * dt
        self.theta += omega * dt

# === HÀM ĐIỀU KHIỂN ===
def go_to_goal_controller(robot, goal, K_v=1.0, K_omega=4.0):
    dx = goal[0] - robot.x
    dy = goal[1] - robot.y
    distance = np.hypot(dx, dy)
    angle_to_goal = np.arctan2(dy, dx)
    angle_diff = angle_to_goal - robot.theta
    angle_diff = np.arctan2(np.sin(angle_diff), np.cos(angle_diff))  # normalize

    v = K_v * distance
    omega = K_omega * angle_diff
    return v, omega, distance

# === MÔ PHỎNG ===
def simulate(goal=(5, 5), T=20, dt=0.1):
    robot = DifferentialDriveRobot()
    frames = int(T / dt)
    trajectory = []

    for _ in range(frames):
        v, omega, d = go_to_goal_controller(robot, goal)
        robot.update(v, omega, dt)
        trajectory.append((robot.x, robot.y, robot.theta))
        if d < 0.05:
            break
    return robot, trajectory

# === ANIMATION HIỂN THỊ TRỰC TIẾP TRÊN COLAB ===
def animate_robot(goal=(5, 5)):
    robot, trajectory = simulate(goal)
    x_data, y_data = [], []

    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.set_xlim(-1, 7)
    ax.set_ylim(-1, 7)
    ax.plot(goal[0], goal[1], 'ro', label='Goal')  # điểm đích
    ax.legend()

    robot_dot, = ax.plot([], [], 'bo', markersize=8)
    path_line, = ax.plot([], [], 'g--', linewidth=1)

    def update(frame):
        x, y, theta = trajectory[frame]
        x_data.append(x)
        y_data.append(y)
        robot_dot.set_data([x], [y])
        path_line.set_data(x_data, y_data)
        return robot_dot, path_line

    ani = FuncAnimation(fig, update, frames=len(trajectory), interval=100)
    plt.close(fig)
    return HTML(ani.to_jshtml())

# === CHẠY VÀ HIỂN THỊ ===
animate_robot(goal=(5, 5))
